# Building a Basic Agent

In [6]:
from llama_index.core.agent import ReActAgent
# from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import FunctionTool

## Create basic tools

In [5]:
def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a*b


# Create a multiply tool
multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a+b

add_tool = FunctionTool.from_defaults(fn=add)


## Initialise LLM

In [7]:
llm = Ollama(model='llama3.2', request_timeout=120)

## Initialise Agent

In [9]:
agent = ReActAgent.from_tools(tools=[multiply_tool, add_tool], llm=llm, verbose=True)

## Testing Agent

In [10]:
response = agent.chat("What is 20+(2*4)? Use a tool to calculate every step.")

> Running step 0ed9f81f-3b1b-4e92-bc91-b1b31f59ae45. Step input: What is 20+(2*4)? Use a tool to calculate every step.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step 9fb479e4-2e86-49c0-89e5-6b460afde029. Step input: None
Thought: I can continue with the calculation. The current equation is 28 + (2 * 4). To complete this, we need to multiply 2 and 4 first.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step 2ce58407-dec4-4083-bb19-816026c36242. Step input: None
Thought: Now that I have the result of the multiplication (8), I can add it to 20 to get the final answer.
Action: add
Action Input: {'a': 28, 'b': 8}
Observation: 36
> Running step 4d1cbdc9-e93d-4a4c-a045-1b6a2f5a3992. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 1e53c067-01bc-456a-

# RAG with Agents

In [28]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

## Load the data

In [23]:
documents = SimpleDirectoryReader('./data/').load_data()
documents[0]

Document(id_='7b95e77b-4c27-4a1b-a14b-c9a711e2c70a', embedding=None, metadata={'page_label': '1', 'file_name': '2023_canadian_budget.pdf', 'file_path': '/Users/jnk789/Developer/LLM/Building-LLMs-with-PyTorch/LlamaIndex/data/2023_canadian_budget.pdf', 'file_type': 'application/pdf', 'file_size': 376126, 'creation_date': '2024-12-26', 'last_modified_date': '2024-12-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='‹\xa02022 2024›\n2023 budget of the\nCanadian federal\ngovernment\nSubmitted 28 March 2023\nPresented 28 March 2023\nParliament 44th\nParty Liberal\nFinance\nminister\nChrystia Freeland\nTotal revenue $456.8 billion\n(projected

## Setup Models

In [29]:
Settings.llm = Ollama(model="llama3.2", timeout=120.0)

In [30]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")

## RAG

In [31]:
index = VectorStoreIndex.from_documents(documents)

In [32]:
query_engine = index.as_query_engine()

In [33]:
response = query_engine.query(
    "What was the total amount of the 2023 Canadian federal budget?"
)
print(response)

$496.9 billion


## Adding Query Engine Tool

In [34]:
from llama_index.core.tools import QueryEngineTool

Now we turn our query engine into a tool by supplying the appropriate metadata (for the python functions, this was being automatically extracted so we didn't need to add it):

In [36]:
budget_tool = QueryEngineTool.from_defaults(
    query_engine=query_engine,
    name="canadian_budget_2023",
    description="A RAG engine with some basic facts about the 2023 Canadian federal budget.",
)

We modify our agent by adding this engine to our array of tools (we also remove the llm parameter, since it's now provided by settings):



In [43]:
agent = ReActAgent.from_tools([add_tool, multiply_tool, budget_tool], verbose=True, max_iterations=20)

## Query the Agent tool

In [45]:
response = agent.query(
    "What is the total amount of Canadian budegt in Indian Rupee. Provided 1 Canadian Dollar equals to 59 Indian Rupee. Go step by step, using a tool to do any math"
)

> Running step a98068c4-22ea-442b-a9a8-ca515bdd6492. Step input: What is the total amount of Canadian budegt in Indian Rupee. Provided 1 Canadian Dollar equals to 59 Indian Ruppe. Go step by step, using a tool to do any math
Thought: The user wants me to calculate the total amount of Canadian budget in Indian Rupees and needs my assistance with this task.
Action: add
Action Input: {'a': 1000, 'b': 59}
Observation: 1059
> Running step cdb80b4a-ea08-4d15-af56-aba867f57e8e. Step input: None
Thought: I used the tool "add" to calculate the total amount of Canadian budget in Indian Rupees and got an observation that the result is 1059.
Action: multiply
Action Input: {'a': 1000, 'b': 59}
Observation: 59000
> Running step 70a3fb34-b8fb-4288-afa2-6be949184495. Step input: None
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.
> Running step 8ff2b071-71da-4a01-93ca-badb19933765. Step input: None
Thought: I made a mistake in using the tool "mult